In [1]:
# Imports
import csv
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [2]:
# Ingest Data
indexData = pd.read_csv('indexData.csv')
indexInfo = pd.read_csv('indexInfo.csv')
indexProcessed = pd.read_csv('indexProcessed.csv')

In [34]:
# Data Description
indexData.describe()

,Open,High,Low,Close,Adj Close,Volume
count,110253.000000,110253.000000,110253.000000,110253.000000,110253.000000,1.102530e+05
mean,7658.515222,7704.372961,7608.000422,7657.545872,7657.351729,1.273975e+09
std,9011.478913,9066.638548,8954.506981,9011.510444,9011.608900,4.315783e+09
min,54.869999,54.869999,54.869999,54.869999,54.869999,0.000000e+00
25%,1855.030029,1864.510010,1843.979980,1855.060059,1854.179565,0.000000e+00
50%,5194.089844,5226.279785,5154.049805,5194.750000,5194.750000,4.329000e+05
75%,10134.299810,10207.820310,10060.349610,10134.830080,10134.830080,1.734314e+08
max,68775.062500,69403.750000,68516.992190,68775.062500,68775.062500,9.440374e+10


In [35]:
indexProcessed.describe()

,Open,High,Low,Close,Adj Close,Volume,CloseUSD
count,104224.000000,104224.000000,104224.000000,104224.000000,104224.000000,1.042240e+05,104224.000000
mean,8015.353334,8063.324234,7962.581120,8014.366642,8014.161269,1.347646e+09,3046.729177
std,9140.563404,9196.575802,9082.767802,9140.609758,9140.720456,4.427662e+09,3747.865623
min,54.869999,54.869999,54.869999,54.869999,54.869999,0.000000e+00,10.204900
25%,2046.887756,2057.213990,2037.185943,2047.506470,2047.358490,0.000000e+00,320.460898
50%,5772.140137,5812.764892,5725.199951,5773.710205,5773.710205,9.529000e+05,1371.598486
75%,10487.377445,10552.179690,10416.092287,10488.622560,10488.622560,2.064676e+08,4383.045241
max,68775.062500,69403.750000,68516.992190,68775.062500,68775.062500,9.440374e+10,18934.376173


In [51]:
indexInfo

,Region,Exchange,Index,Currency
0,United States,New York Stock Exchange,NYA,USD
1,United States,NASDAQ,IXIC,USD
2,Hong Kong,Hong Kong Stock Exchange,HSI,HKD
3,China,Shanghai Stock Exchange,000001.SS,CNY
4,Japan,Tokyo Stock Exchange,N225,JPY
5,Europe,Euronext,N100,EUR
6,China,Shenzhen Stock Exchange,399001.SZ,CNY
7,Canada,Toronto Stock Exchange,GSPTSE,CAD
8,India,National Stock Exchange of India,NSEI,INR
9,Germany,Frankfurt Stock Exchange,GDAXI,EUR


In [16]:
print(f"length of processed data -> {len(indexProcessed[['Index' , 'Date']].drop_duplicates())}")
print(f"length of unprocessed data -> {len(indexData[['Index' , 'Date']].drop_duplicates())}")

length of processed data -> 104224
length of unprocessed data -> 112457


In [29]:
# Check if unprocessed can be discarded?
df_p = indexProcessed[['Index' , 'Date']].drop_duplicates()
df_p['both'] = df_p['Index'].astype('str')+df_p['Date'].astype('str')
df_p_set = set(df_p['both'])

# Check if unprocessed can be discarded?
df_up = indexData[['Index' , 'Date']].drop_duplicates()
df_up['both'] = df_up['Index'].astype('str')+df_up['Date'].astype('str')
df_up_set = set(df_up['both'])

# len(set(df_up['both']).intersection(set(df_p['both'])))
len(df_up_set - df_p_set)

8233

In [52]:
# Stuff that's been excluded from processed data - Why?
removed_data = indexData.join(df_up[df_up['both'].isin(df_up_set-df_p_set)]['both'] , how = 'inner')
removed_data = removed_data.iloc[:,:-1]
removed_data.dropna()

# Why was the Korean stock exchange removed?

,Index,Date,Open,High,Low,Close,Adj Close,Volume
70042,KS11,1996-12-11,705.989990,709.479980,704.429993,704.679993,704.679993,28000.0
70043,KS11,1996-12-12,705.109985,706.010010,688.739990,689.380005,689.380005,25900.0
70044,KS11,1996-12-13,690.440002,695.719971,677.640015,689.070007,689.070007,26500.0
70045,KS11,1996-12-16,686.969971,686.969971,667.710022,673.919983,673.919983,22800.0
70046,KS11,1996-12-17,675.349976,680.090027,660.390015,663.349976,663.349976,31600.0
...,...,...,...,...,...,...,...,...
76218,KS11,2021-05-26,3171.909912,3183.969971,3160.300049,3168.429932,3168.429932,1269000.0
76219,KS11,2021-05-27,3170.840088,3173.860107,3142.370117,3165.510010,3165.510010,1081100.0
76220,KS11,2021-05-28,3172.090088,3198.659912,3171.850098,3188.729980,3188.729980,966000.0
76221,KS11,2021-05-31,3192.060059,3204.290039,3179.870117,3203.919922,3203.919922,1123100.0


In [54]:
indexProcessed[indexProcessed['Index']=='KS11']

# Need to investigate this further!

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
